<a href="https://colab.research.google.com/github/hardik1057/IPL_Match_prediction/blob/main/IPL_Winning_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Importing Libraries

In [2]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 4.9 MB/s eta 0:00:00


In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.preprocessing import LabelEncoder


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Reading Dataset

In [5]:
df=pd.read_excel("/content/drive/MyDrive/research/Minor Project /test.xlsx")

In [6]:
df.head()

,venue,team1,team2,toss_winner,toss_decision,winner
0,M Chinnaswamy Stadium,RCB,KKR,RCB,FIELD,KKR
1,"Punjab Cricket Association Stadium, Mohali",PBKS,CSK,CSK,BAT,CSK
2,Feroz Shah Kotla,DC,RR,RR,BAT,DC
3,Wankhede Stadium,MI,RCB,MI,BAT,RCB
4,Eden Gardens,KKR,Deccan Chargers,Deccan Chargers,BAT,KKR


##Filtering Dataset

In [7]:
# Given team list
given_team_list = ['MI', 'CSK', 'RCB' , 'DC' , 'GT' , 'LSG' , 'PBKS' , 'RR' , 'KKR' , 'SRH']

# Create a boolean mask for rows where team1 or team2 is not in the given team list
mask = ((df['team1'].isin(given_team_list)) & (df['team2'].isin(given_team_list)))

# Filter out rows based on the mask
filtered_df = df[mask]
df = filtered_df

In [8]:
df.tail()

,venue,team1,team2,toss_winner,toss_decision,winner
822,M Chinnaswamy Stadium,RCB,GT,GT,BOWL,GT
823,"MA Chidambaram Stadium, Chepauk",GT,CSK,GT,BOWL,CSK
824,"MA Chidambaram Stadium, Chepauk",LSG,MI,MI,BAT,MI
825,Narendra Modi Stadium,GT,MI,MI,BOWL,GT
826,Narendra Modi Stadium,CSK,GT,CSK,BOWL,CSK


##Statistical measures of the dataset

In [9]:
print(df.shape)
print(df.size)
print(df.count)
df.describe()
df['team1'].value_counts()
print(df.columns)

(684, 6)
4104
<bound method DataFrame.count of                                           venue team1 team2 toss_winner  \
0                         M Chinnaswamy Stadium   RCB   KKR         RCB   
1    Punjab Cricket Association Stadium, Mohali  PBKS   CSK         CSK   
2                              Feroz Shah Kotla    DC    RR          RR   
3                              Wankhede Stadium    MI   RCB          MI   
5                        Sawai Mansingh Stadium    RR  PBKS        PBKS   
..                                          ...   ...   ...         ...   
822                       M Chinnaswamy Stadium   RCB    GT          GT   
823             MA Chidambaram Stadium, Chepauk    GT   CSK          GT   
824             MA Chidambaram Stadium, Chepauk   LSG    MI          MI   
825                       Narendra Modi Stadium    GT    MI          MI   
826                       Narendra Modi Stadium   CSK    GT         CSK   

    toss_decision winner  
0           FIELD    KKR 

##LabelEncoding the dataset

In [12]:
# Create an instance of LabelEncoder
lab = LabelEncoder()
# Perform label encoding on the 'my_column' column
df['team1'] = lab.fit_transform(df['team1'])
df['team2'] = lab.fit_transform(df['team2'])
df['toss_winner'] = lab.fit_transform(df['toss_winner'])
df['toss_decision'] = lab.fit_transform(df['toss_decision'])
df['venue'] = lab.fit_transform(df['venue'])
df['winner'] = lab.fit_transform(df['winner'])
print(df)

     venue  team1  team2  toss_winner  toss_decision  winner
0       11      7      3            7              2       3
1       15      6      0            0              0       0
2        6      1      8            8              0       1
3       23      5      7            5              0       8
5       19      8      6            6              0       9
..     ...    ...    ...          ...            ...     ...
822     11      7      2            2              1       2
823     12      2      0            2              1       0
824     12      4      5            5              0       5
825     14      2      5            5              1       2
826     14      0      2            0              1       0

[684 rows x 6 columns]


In [14]:
X = df.drop(columns=['winner'],axis=1)
Y = df['winner']
print(X)
print(Y)

     venue  team1  team2  toss_winner  toss_decision
0       11      7      3            7              2
1       15      6      0            0              0
2        6      1      8            8              0
3       23      5      7            5              0
5       19      8      6            6              0
..     ...    ...    ...          ...            ...
822     11      7      2            2              1
823     12      2      0            2              1
824     12      4      5            5              0
825     14      2      5            5              1
826     14      0      2            0              1

[684 rows x 5 columns]
0      3
1      0
2      1
3      8
5      9
      ..
822    2
823    0
824    5
825    2
826    0
Name: winner, Length: 684, dtype: int64


#KNN

In [15]:
knn = KNeighborsClassifier(n_neighbors=5)
X_train,X_test,Y_train,Y_test = train_test_split(X,Y, test_size=0.2, random_state=1) #stratify=Y
knn.fit(X_train, Y_train)
y_pred = knn.predict(X_test)
accuracy = accuracy_score(y_pred, Y_test)
print(f"Accuracy score of test data: {accuracy:.2f}")
train_pred = knn.predict(X_train)
accuracy = accuracy_score(train_pred, Y_train)
print("Accuracy score of train data = {}".format(accuracy))

Accuracy score of test data: 0.41
Accuracy score of train data = 0.5904936014625228


##Decision Tree

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
tree_classifier = DecisionTreeClassifier(max_depth=11, criterion='gini')
tree_classifier.fit(X_train, y_train)
train_pred = tree_classifier.predict(X_train)
accuracy = accuracy_score(train_pred, y_train)
print("Accuracy score of train data = {}".format(accuracy))
y_pred = tree_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of test data: {accuracy:.2f}")

Accuracy score of train data = 0.8117001828153565
Accuracy of test data: 0.58


##Random Forest

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
rf_classifier = RandomForestClassifier(n_estimators=100, max_depth=11, random_state=42)
rf_classifier.fit(X_train, y_train)
train_pred = rf_classifier.predict(X_train)
accuracy = accuracy_score(train_pred, y_train)
print("Accuracy score of train data = {}".format(accuracy))
y_pred = rf_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of test data: {accuracy:.2f}")

Accuracy score of train data = 0.8537477148080439
Accuracy of test data: 0.53


##SVM Polynomial

In [18]:
classifier = svm.SVC(kernel='poly' , degree =7 , gamma='scale')
classifier.fit(X_train, y_train)
train_pred = classifier.predict(X_train)
accuracy = accuracy_score(train_pred, y_train)
print("Accuracy score of train data = {}".format(accuracy))
test_pred = classifier.predict(X_test)
accuracy2 = accuracy_score(test_pred, y_test)
print('Accuracy score on test data = {}'.format(accuracy2))

Accuracy score of train data = 0.603290676416819
Accuracy score on test data = 0.40145985401459855


##SVM RBF

In [19]:
classifier = svm.SVC(kernel='rbf', gamma='scale')
classifier.fit(X_train, y_train)
train_pred = classifier.predict(X_train)
accuracy = accuracy_score(train_pred, y_train)
print("Accuracy score of train data = {}".format(accuracy))
test_pred = classifier.predict(X_test)
accuracy2 = accuracy_score(test_pred, y_test)
print('Accuracy score on test data = {}'.format(accuracy2))

Accuracy score of train data = 0.4570383912248629
Accuracy score on test data = 0.38686131386861317


##LightGBM

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
lgbm_classifier = LGBMClassifier(boosting_type='gbdt', num_leaves=4, learning_rate=0.1, n_estimators=200)
lgbm_classifier.fit(X_train, y_train)
train_pred = lgbm_classifier.predict(X_train)
accuracy = accuracy_score(train_pred, y_train)
print("Accuracy score of train data = {}".format(accuracy))
y_pred = lgbm_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy score of test data: {accuracy:.2f}")


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000325 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 57
[LightGBM] [Info] Number of data points in the train set: 547, number of used features: 5
[LightGBM] [Info] Start training from score -2.041769
[LightGBM] [Info] Start training from score -2.334157
[LightGBM] [Info] Start training from score -3.126395
[LightGBM] [Info] Start training from score -2.145566
[LightGBM] [Info] Start training from score -3.906554
[LightGBM] [Info] Start training from score -1.885608
[LightGBM] [Info] Start training from score -2.244006
[LightGBM] [Info] Start training from score -6.304449
[LightGBM] [Info] Start training from score -2.130062
[LightGBM] [Info] Start training from score -2.226911
[LightGBM] [Info] Start training from score -2.279097
[LightGBM] [Info] Start training from score -5.611302
[Lig

##Naive Bayes

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
nb_classifier = GaussianNB()
nb_classifier.fit(X_train, y_train)
train_pred = nb_classifier.predict(X_train)
accuracy = accuracy_score(train_pred, y_train)
print("Accuracy score of train data = {}".format(accuracy))
y_pred = nb_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy score of test data: {accuracy:.2f}")

Accuracy score of train data = 0.3875685557586837
Accuracy score of test data: 0.31


##Cat Boost

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=41)
model = CatBoostClassifier(iterations=100, depth=4, learning_rate=0.1, loss_function='MultiClass', verbose=False)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of test data: {accuracy:.2f}")
train_pred = model.predict(X_train)
accuracy = accuracy_score(train_pred, y_train)
print("Accuracy score of train data = {}".format(accuracy))

Accuracy of test data: 0.55
Accuracy score of train data = 0.6654478976234004
